In [10]:
import json
import requests
import time
from collections import Counter

# Riot API 키 설정
api_key = 'RGAPI-139be212-60b2-448a-a0c5-2537fe2fd3f3'  # 유효한 Riot API 키로 교체하세요.

# 요청 헤더 설정
request_header = {
    "User-Agent": "Mozilla/5.0",
    "Accept-Language": "ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7",
    "Accept-Charset": "application/x-www-form-urlencoded; charset=UTF-8",
    "Origin": "https://developer.riotgames.com",
    "X-Riot-Token": api_key
}

# TFT 매치 리스트를 가져오는 함수
def get_tft_match_ids(puuid, count=10):
    match_list_url = f"https://asia.api.riotgames.com/tft/match/v1/matches/by-puuid/{puuid}/ids?count={count}"
    try:
        response = requests.get(match_list_url, headers=request_header)
        response.raise_for_status()
        match_ids = response.json()
        return match_ids
    except requests.exceptions.RequestException as e:
        print(f"Error fetching match list for PUUID {puuid}: {e}")
        return []

# 매치 ID를 사용하여 매치 상세 정보를 가져오는 함수
def get_match_details(match_id):
    match_detail_url = f"https://asia.api.riotgames.com/tft/match/v1/matches/{match_id}"
    try:
        response = requests.get(match_detail_url, headers=request_header)
        response.raise_for_status()
        match_details = response.json()
        return match_details
    except requests.exceptions.RequestException as e:
        print(f"Error fetching match details for Match ID {match_id}: {e}")
        return None

# 시너지 정보를 추출하는 함수
def extract_synergies(match_details, puuid):
    synergies = []
    if match_details and 'info' in match_details:
        participants = match_details['info'].get('participants', [])
        for participant in participants:
            if participant.get('puuid') == puuid:
                traits = participant.get('traits', [])
                for trait in traits:
                    if trait.get('tier_current', 0) > 0:  # 활성화된 시너지
                        synergies.append(trait.get('name'))
    return synergies

# tft_player_results.json 파일 로드
with open('tft_played_results.json', 'r', encoding='utf-8') as file:
    tft_played_results = json.load(file)

# TFT를 플레이한 PUUID 목록 추출
played_puuids = [puuid for puuid, played in tft_played_results['details'].items() if played]

# 전체 시너지 빈도수를 저장할 Counter 객체
total_synergy_counter = Counter()

# 각 PUUID에 대해 매치 데이터 분석
for puuid in played_puuids:
    match_ids = get_tft_match_ids(puuid, count=10)  # 최근 10게임의 매치 ID 가져오기
    for match_id in match_ids:
        match_details = get_match_details(match_id)
        synergies = extract_synergies(match_details, puuid)
        total_synergy_counter.update(synergies)
        # API 호출 제한을 피하기 위해 지연 시간 추가
        time.sleep(1.2)  # 1.2초 지연

# 가장 많이 사용된 시너지 덱 상위 10개 출력
most_common_synergies = total_synergy_counter.most_common(1)
print("가장 많이 사용된 시너지 덱 상위 10개:")
for synergy, count in most_common_synergies:
    print(f"{synergy}: {count}회 사용")

# 결과를 JSON 파일로 저장
with open('synergy_usage_results.json', 'w', encoding='utf-8') as file:
    json.dump(total_synergy_counter, file, ensure_ascii=False, indent=4)

print("시너지 사용 빈도 분석이 완료되었습니다.")


KeyboardInterrupt: 

In [14]:
import requests
import json
import time
from collections import Counter

# Riot API 키 설정
api_key = 'RGAPI-139be212-60b2-448a-a0c5-2537fe2fd3f3'  # 유효한 Riot API 키로 교체하세요.
conunt = 1
# 요청 헤더 설정
request_header = {
    "User-Agent": "Mozilla/5.0",
    "Accept-Language": "ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7",
    "Accept-Charset": "application/x-www-form-urlencoded; charset=UTF-8",
    "Origin": "https://developer.riotgames.com",
    "X-Riot-Token": api_key
}

# PUUID 목록 로드
with open('puuid_list.json', 'r', encoding='utf-8') as file:
    puuid_list = json.load(file)

# TFT 매치 리스트를 가져오는 함수
def get_tft_match_ids(puuid, count=10):
    match_list_url = f"https://asia.api.riotgames.com/tft/match/v1/matches/by-puuid/{puuid}/ids?count={count}"
    try:
        response = requests.get(match_list_url, headers=request_header)
        response.raise_for_status()
        match_ids = response.json()
        return match_ids
    except requests.exceptions.RequestException as e:
        print(f"Error fetching match list for PUUID {puuid}: {e}")
        return []

# 매치 ID를 사용하여 매치 상세 정보를 가져오는 함수
def get_match_details(match_id):
    match_detail_url = f"https://asia.api.riotgames.com/tft/match/v1/matches/{match_id}"
    try:
        response = requests.get(match_detail_url, headers=request_header)
        response.raise_for_status()
        match_details = response.json()
        return match_details
    except requests.exceptions.RequestException as e:
        print(f"Error fetching match details for Match ID {match_id}: {e}")
        return None

# 특정 플레이어의 매치에서 가장 많이 활성화된 시너지를 추출하는 함수
def get_top_synergy(match_details, puuid):
    if match_details and 'info' in match_details:
        participants = match_details['info'].get('participants', [])
        for participant in participants:
            if participant.get('puuid') == puuid:
                # 시너지 리스트에서 num_units가 가장 큰 시너지 추출
                traits = participant.get('traits', [])
                if not traits:
                    return None
                # num_units 기준으로 정렬하고 가장 높은 값 추출
                top_synergy = max(traits, key=lambda t: t.get('num_units', 0))
                return {
                    "synergy_name": top_synergy.get('name'),
                    "num_units": top_synergy.get('num_units', 0)
                }
        print(count)
        count += 1
    return None

# 전체 결과 저장
top_synergies = []

# 각 PUUID에 대해 매치 데이터 분석
for puuid in puuid_list:
    match_ids = get_tft_match_ids(puuid, count=10)  # 최근 10게임의 매치 ID 가져오기
    for match_id in match_ids:
        match_details = get_match_details(match_id)
        top_synergy = get_top_synergy(match_details, puuid)
        if top_synergy:
            top_synergies.append({
                "puuid": puuid,
                "match_id": match_id,
                "synergy_name": top_synergy["synergy_name"],
                "num_units": top_synergy["num_units"]
            })
        # API 호출 제한을 피하기 위해 지연 시간 추가
        time.sleep(1.2)  # 1.2초 지연

# 결과를 JSON 파일로 저장
with open('top_synergies_results.json', 'w', encoding='utf-8') as file:
    json.dump(top_synergies, file, ensure_ascii=False, indent=4)

print("활성화된 시너지 분석이 완료되었습니다. 결과가 'top_synergies_results.json'에 저장되었습니다.")

Error fetching match details for Match ID KR_7378808113: 503 Server Error: Service Unavailable for url: https://asia.api.riotgames.com/tft/match/v1/matches/KR_7378808113
활성화된 시너지 분석이 완료되었습니다. 결과가 'top_synergies_results.json'에 저장되었습니다.
